# COUNT THE OCCURRENCE OF A PARTICULAR WORD IN EACH AND ALL REVIEWS

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import pandas as pd

In [7]:
file_dataset = "df_processed_polarity_and_satisfaction.csv"
# file_out = "df_processed_sentimentValue.csv"

In [8]:
df = pd.read_csv(file_dataset)
df.head()

,_id,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,ProductURL,Year,Month,Day,Polarity,Customer_Satisfied
0,0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,2011-04-27,Good Quality Dog Food,I have bought several of the Vitality canned d...,https://www.amazon.com/dp/B001E4KFG0,2011,4,27,0.9441,1
1,2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,2008-08-18,"""Delight"" says it all",This is a confection that has been around a fe...,https://www.amazon.com/dp/B000LQOCH0,2008,8,18,0.8265,1
2,3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,2011-06-13,Cough Medicine,If you are looking for the secret ingredient i...,https://www.amazon.com/dp/B000UA0QIQ,2011,6,13,0.0000,0
3,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,2012-09-07,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,https://www.amazon.com/dp/B00813GRG4,2012,9,7,-0.5664,0
4,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,2012-10-21,Great taffy,Great taffy at a great price. There was a wid...,https://www.amazon.com/dp/B006K2ZZ7K,2012,10,21,0.9468,1


In [9]:
# Reviews
reviews = df[['Id', 'Text']]

In [10]:
#Corpus of original reviews
reviews_corpus = reviews['Text'].tolist()

## Raw Count

In [11]:
#Raw word count
vectorizer_raw = CountVectorizer()
raw_counts = vectorizer_raw.fit_transform(reviews_corpus)

In [12]:
feature_names = list(vectorizer_raw.get_feature_names_out())

In [13]:
len(feature_names)

120246

## Count with Preprocessing

### Preprocessing Functions

In [ ]:
#!pip install html5lib

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import string
import re
from urllib.parse import urlparse

from bs4 import BeautifulSoup

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
#Remove Stop Words
def remove_stopwords(review):
  words = word_tokenize(review)
  words = [word for word in words if word.lower() not in stop_words]
  processed_review = ' '.join(words)

  return processed_review

In [ ]:
#Remove Punctuation
def remove_punctuation(review):
  processed_review = review.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
  return processed_review

In [ ]:
#Remove numbers
def remove_numbers(review):
  processed_review = ''.join(character for character in review if not character.isdigit())
  return processed_review

In [ ]:
#Lowercase sentence
def lowercase_text(text):
    return text.lower()

In [ ]:
#Remove Links
def remove_links(review):
  words = review.split()
  for i, word in enumerate(words):
    parsed_url = urlparse(word)
    if parsed_url.scheme and parsed_url.netloc:
      words[i] = ""

  return ' '.join(words)

In [ ]:
#Remove html expressions
def remove_html(review):
  process = BeautifulSoup(review, 'html5lib')
  processed_review = process.get_text(separator=" ")
  return processed_review

In [ ]:
def preprocess_review(review):
  preprocessed_review = remove_html(review)
  preprocessed_review = remove_links(preprocessed_review)
  preprocessed_review = remove_punctuation(preprocessed_review)
  preprocessed_review = remove_stopwords(preprocessed_review)
  preprocessed_review = remove_numbers(preprocessed_review)
  preprocessed_review = lowercase_text(preprocessed_review)

  return preprocessed_review

### Preprocess the Corpus

In [ ]:
%%time
#Preprocess the whole corpus
#Note: 10+ minutes on Colab
preprocessed_corpus = [preprocess_review(review) for review in reviews_corpus]

<ipython-input-22-eb3c6563d70c>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  process = BeautifulSoup(review, 'html5lib')


CPU times: user 16min 22s, sys: 3.36 s, total: 16min 25s
Wall time: 17min


In [ ]:
#Turn array into dataframe
processed_text_dataframe = pd.DataFrame(preprocessed_corpus, columns = ['Processed Text'])

#Add review Ids to the dataframe
processed_text_dataframe = pd.concat([reviews["Id"], processed_text_dataframe], axis=1)

In [ ]:
processed_text_dataframe.to_csv('processed_text_dataframe.csv', index=False)

In [ ]:
processed_text_dataframe

,Id,Processed Text
0,1,bought several vitality canned dog food produc...
1,2,product arrived labeled jumbo salted peanuts p...
2,3,confection around centuries light pillowy citr...
3,4,looking secret ingredient robitussin believe f...
4,5,great taffy great price wide assortment yummy ...
...,...,...
568372,568449,complaint much use huge amount spice gave ja...
568373,568451,disappointed flavor chocolate notes especially...
568374,568452,stars small give one training session tried ...
568375,568453,best treats training rewarding dog good groomi...


### Count the ocurrences of each term for each and all reviews

In [ ]:
#Processed Word Count
vectorizer_processed = CountVectorizer()
processed_counts = vectorizer_processed.fit_transform(preprocessed_corpus)

In [ ]:
#Processed Vocabulary
feature_names_processed = list(vectorizer_processed.get_feature_names_out())

In [ ]:
#Save processed vocabulary
#vocab_df = pd.DataFrame(feature_names_processed, columns=['Term'])
#vocab_df.to_csv('vocabulary_processed.csv', index=False)

In [ ]:
len(feature_names_processed)

106831

# Saving the Term Count Per Review

In [ ]:
import numpy as np

## Saving the counts for all reviews

In [ ]:
#This code shoud work if there is enough RAM available
def save_term_counts_csv():
  term_counts_array = processed_counts.toarray()

  #Turn the array into a pandas dataframe
  term_counts_dataframe = pd.DataFrame(term_counts_array, columns=feature_names_processed)

  #Append the review ID at the beginning for cross referencing
  term_counts_dataframe = pd.concat([reviews["Id"], term_counts_dataframe], axis=1)

  #Save the review as
  term_counts_dataframe.to_csv('term_counts_per_review.csv', index=False)

## Save the term counts of a particular set of reviews

In [ ]:
def get_review_term_counts(review_ids, filename):
  #List the reviews' row Ids
  index_list = [int(dataset.index[dataset['Id'] == id][0]) for id in review_ids]

  #Get a list of terms that are in at least one of the reviews
  vocab_id_list = []

  for ind in index_list:
    review_vocab = processed_counts[ind, :]
    vocab_ids = [vocab_id for vocab_id in review_vocab.nonzero()[1]]

    for vocab_id in vocab_ids:
      if vocab_id not in vocab_id_list:
        vocab_id_list.append(vocab_id)

  #For each term, get the count of it in each review
  id_term_counts = {}

  #First put the review Ids
  id_term_counts['Id'] = review_ids

  for vocab_id in vocab_id_list:
    vocab_counts = []

    for ind in index_list:
      vocab_counts.append(processed_counts[int(ind), int(vocab_id)])

    id_term_counts[feature_names_processed[int(vocab_id)]] = vocab_counts

  #Turn into a pandas DataFrame
  counts_dataframe = pd.DataFrame.from_dict(id_term_counts)

  counts_dataframe.to_csv(f'{filename}.csv', index=False)

  return(counts_dataframe)


In [ ]:
#Test
get_review_term_counts([69726], 'review_69726')

,Id,product,highly,recommend,flavor,bag,black,one,really,never,...,herbal,result,brew,traditional,stated,darjeeling,tolerate,peach,genuine,fruitty
0,69726,1,1,1,2,1,3,1,1,1,...,1,1,1,1,1,1,1,4,1,1
